## Data Structures: Memory Allocation and Lists

> Changing a data structure in a slow program can work the same way an organ transplant does in a sick patient. Important classes of abstract data types such as containers, dictionaries, and priority queues, have many different but functionally equivalent data structures that implement them. Changing the data structure does not change the correctness of the program, since we presumably replace a correct implementation with a different correct implementation. However, the new implementation mentation of the data type realizes different tradeoffs in the time to execute various operations, so the total performance can improve dramatically. Like a patient in need of a transplant, only one part might need to be replaced in order to fix the problem.

Steven S Skiena. The Algorithm Design Manual 

## Process address space

Let's start by understanding what we mean when we say a program is "in memory." As you've already learned, a C program is compiled to an executable binary, which contains the instructions that the machine will execute. In order to actually run the program, this compiled code must be loaded from disk into memory. In addition, once your program starts, it must create space for the variables it will use, and it must store and read values from those variables. The code, the reserved space, and the generated data all constitute a program's memory footprint.



Every operating system has a (different) convention for exactly where and how these different resources are actually laid out in memory. (These conventions are called object file formats, where "object file" in this context refers to the semi-finished intermediate files that a compiler produces, not to be confused with objects as instances of class data types in languages like Python or C++.) In Linux, the most common object format is called ELF, short for "Executable and Linkable Format", and we'll use that as an example here, but most object file formats have very similar concepts.

By using program analysis tools, we can actually take a sample program and compute the (relative) addresses for all its different pieces. A simplified view of our example program looks like this in memory:the stack and the heap

![](https://raw.githubusercontent.com/iacs-cs207/cs207/gh-pages/lectures/f01/segments.png)

### The Stack

The stack keeps track of function calls and their data. Whenever a function is called, a new chunk of memory is reserved at the top of the stack in order to store variables for that function. This includes variables that you have defined inside your function and function parameters, but it also includes data that were generated automatically by the compiler. The most recognizable value in this latter category is the return address. When a function calls return, the computer starts executing instructions at the location the function was originally called from, and the return address is what lets the computer know where it left off. Additionally, when a function returns, it removes its stack frame from the stack. This means that at any given point, the stack contains a record of which functions the program is currently in. When you run backtrace in gdb, this is exactly the information it's telling you.

### The Heap

This is a problem if you want to create space for variables and then access them after the function returns. What we really need is another memory location which is not reclaimed after a function returns. That's what the heap is for. The heap is explicitly managed, which means that in order to allocate a variable in it, you need to ask. What's more, when you're finished with that space, you need to say so. This interface, explicitly requesting and releasing memory from the heap, is typically called memory management.



### Simple arrays and memory management in C

In [338]:
%%file simple.c
#include <stdlib.h>
#include <stdio.h>

int* just_a_func(){
    int *heapintlist;
    int i;
    int stackintlist[3]; /*Allocate in this functions frame on stack*/
    heapintlist = (int *) malloc(3*sizeof(int)); /*Allocate on heap*/
    for (i = 0; i < 3; i++) {
        stackintlist[i] = i;
        heapintlist[i] =i;
    }
    for (i = 0; i < 3; i++) {
        printf("stackintlist %d\n",stackintlist[i]);
        printf("heapintlist %d :  %d\n", i, heapintlist[i]);
    }
    return heapintlist; /*stackintlist unallocated here */
}
int main() {
    int* heapintlist;
    int i;
    
    heapintlist=just_a_func();
    for (i = 0; i < 3; i++) {
        heapintlist[i] =i+5;
        printf("heapintlist %d :  %d\n", i, heapintlist[i]);
    }
    free(heapintlist);
    /* garbage in &heapinlist[i] after this */
}

Overwriting simple.c


In [339]:
!gcc -o simple -g simple.c

In [340]:
!./simple

stackintlist 0
heapintlist 0 :  0
stackintlist 1
heapintlist 1 :  1
stackintlist 2
heapintlist 2 :  2
heapintlist 0 :  5
heapintlist 1 :  6
heapintlist 2 :  7


#### If you malloc, you must free

Control over memory allocation is a powerful tool, but it comes with responsibilities. While the operating system will reclaim all the memory from a process after it exits (including the heap, any stack space, and all program segments), the programmer is completely responsible during the program's lifetime. So calling free on memory when you are finished with it is not particularly optional. For instance, let's say you have a web server program, and every time a page is requested, you allocated some memory on the heap but forgot to free it. As time goes on, more and more memory is reserved on the heap for you. Because C will not automatically free it up, the heap will begin to expand. Eventually, the OS will run out of physical RAM, and it will either start swapping (which is when it tries to start using disk as a back-up for memory---a death knell for performance) or simply start killing random programs to free up memory.

## How does Python manage memory for lists?

In python, when you create a list, like so:

In [2]:
a=[1,2,3,4,5]

the layout in memory you get is not what you might have expected. For example, you might have expected 5 continuous integers, with some book-keeping at the beginning, either on the stack or the heap. Such a data structure is called an array, but the above is not an array of integers. 

#### Python objects are allocated on the heap

What is it? To understand this we must first understand how an int is represented in Python. It is represented as a C structure **allocated on the heap**/

The picture of this C structure looks a little bit  like this (taken from Jake Vanderplas's excellent [Why Python is Slow](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/):

![](http://jakevdp.github.io/images/cint_vs_pyint.png)

On Jake's blog, he shows how this integer is represented:

```C
struct _longobject {
    long ob_refcnt; // in PyObject_HEAD
    PyTypeObject *ob_type; // in PyObject_HEAD
    size_t ob_size; // in PyObject_HEAD
    long ob_digit[1];
};
```

Notice a reference count `ob_refcnt`? We saw earlier how the `id`'s of two variables having the same value, say 10, are the same. That id points to this structure. Its preallocated by python and kept around. Jake shows us how to poke at it..

In [341]:
import ctypes

class IntStruct(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_digit", ctypes.c_long)]
    
    def __repr__(self):
        return ("IntStruct(ob_digit={self.ob_digit}, "
                "refcount={self.ob_refcnt})").format(self=self)
num = 5
IntStruct.from_address(id(num)) # 5 is a struct allocated on heap and pointed to 200+ times!

IntStruct(ob_digit=5, refcount=259)

A python list is represented by this struct:

```C
typedef struct {
    long ob_refcnt;
    PyTypeObject *ob_type;
    Py_ssize_t ob_size;
    PyObject **ob_item;
    long allocated;
} PyListObject;
```

Notice the `PyObject**`. This points to the contents of the list. What is this a list of? This is a list of `PyObject*`.  Each of those pointers when dereferenced gives us one of the integer structure above. The `ob_size` tells us the number of items on the list.

Thus this is **an array of pointers to heap allocated inststruct**s.

This is illustrated next (int 1 means an instruct with digit 1)

In [343]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=def+b(%29%3A%0A++++a%3D%5B1,2,3,4,5%5D%0Aa%3D%5B1,2,3,4,5%5D%0Ab(%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=true&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=0&codeDivWidth=350&codeDivHeight=400"> </iframe>')

But wait. In Python you can append to lists. Ad Infinitum (well upto VM memory). Whats a `ob_size` doing in our struct then? Turns out Python lists are implemented in something called a dynamic array!

### Arrays

To talk about dynamic arrays, we ought to first define a static array. This is a contiguous slab of memory of known size, such that n items (ints ot structs or whatever) can fit in. This is a great data structure. Why?

- constant time index access: a[i] is O(1)...just seek i*sizeof(int) for example down
- linear time traversal or search: 1 unit per loop iteration means O(n) in loop.
- locality in memory: its one int after another

Tuples in Python are fixed size, static arrays..



But the big problem is, what if I want to add something more beyond the end of the array? Then we fall flat, so we use..

### Dynamic Arrays

What python does is first create a fixed size array of these `Pyobject*` pointers on the heap. Then, as you append, it uses its own algorithm to figure out when to expand the size of the array.

In [7]:
alist=[1,2,3,4]
alist.append(5)
alist

[1, 2, 3, 4, 5]

From https://svn.python.org/projects/python/trunk/Objects/listobject.c :

```C
/* This over-allocates proportional to the list size, making room
     * for additional growth.  The over-allocation is mild, but is
     * enough to give linear-time amortized behavior over a long
     * sequence of appends() in the presence of a poorly-performing
     * system realloc().
     * The growth pattern is:  0, 4, 8, 16, 25, 35, 46, 58, 72, 88, ...
     */
new_allocated = (newsize >> 3) + (newsize < 9 ? 3 : 6);
```

In [9]:
import sys
sys.getsizeof([]),sys.getsizeof([1]),sys.getsizeof([1,1]), sys.getsizeof([1,1,1])

(64, 72, 80, 88)

In [344]:
a=[]
sys.getsizeof(a)

64

In [345]:
a.append(1)
sys.getsizeof(a)

96

In [346]:
a.append(1)
sys.getsizeof(a)

96

In [347]:
for j in range(16):
    a.append(1)
    print(a)
    print(sys.getsizeof(a))

[1, 1, 1]
96
[1, 1, 1, 1]
96
[1, 1, 1, 1, 1]
128
[1, 1, 1, 1, 1, 1]
128
[1, 1, 1, 1, 1, 1, 1]
128
[1, 1, 1, 1, 1, 1, 1, 1]
128
[1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
192
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
264
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
264


### Dynamic Arrays performance

Lets assume we start with a size of 1 and then double the size each time. It then takes $lg(n)$ doublings for the array to have n slots.

The last n/2 numbers wouldnt move at all. The previous n/4 would have moved once, the previous n/8 twice, and so on

Thus the total number of movements is 

$$ \sum_{i=1}^{lg(n)} i*\frac{n}{2^{i+1}} \le \frac{n}{2} \sum_{i=1}^{\infty} \frac{i}{2^i} = n$$

This is an amazing result. The work of reallocation is still $O(n)$ on the average, as if a single array had been allocated in advance! 

#### Dynamic Arrays in C

Python Dynamic arrays are implemented in C.

Let us make a toy implementation (based off an example from The Practice of Programming) which makes a dynamic array of integers. There is a canonical way to write such programs:

- a "header" file (`dynamic_array.h`) which provides an interface
- a "c" file (`dynamic_array.c`) which provides an implementation
- a "driver" file, with a main function(`dademo.c`), that uses this implementation

We first write a header file which talks about the interface we want to implement.

- We construct a struct, with `upto` marking the index to which we have actual values
- `size` marks the actual amount of space allocated for the integers
- array is a pointer to an array of ints, allocated on the heap, that we use for our implementation
- we allocate on the heap to persist our data thru multiple function allocations
- `append` if the function on which reallocation will occur

In [69]:
%%file dynamic_array.h


typedef struct DArray {
    int upto;
    int size;
    int *array;
} DArray;

int DArray_init(DArray* arr);
int append(DArray* arr, int i);
int get(DArray* arr, int index);
int get_index(DArray* arr, int value);
int set(DArray* arr, int index, int value);
int delete(DArray* arr, int value);
void DArray_free(DArray* arr);

Overwriting dynamic_array.h


In [348]:
%%file dynamic_array.c


#include <stdlib.h>
#include <string.h>
#include "dynamic_array.h"

int CAPACITY_INIT = 1;
int GROWTH_FACTOR = 2;

int DArray_init(DArray* arr){
    arr->array = (int *) malloc(CAPACITY_INIT*sizeof(int));
    if (arr->array == NULL) {
        return -1;
    }
    arr->upto = 0;
    arr->size = CAPACITY_INIT;
    return 1;
}

int append(DArray* arr, int i){
    int *iptr;
    if (arr->upto >= arr->size) {
        /* tmp pointer needed below as if allocation failed, original array would be lost */
        iptr = (int *) realloc(arr->array, arr->size*GROWTH_FACTOR*sizeof(int));
        if (iptr == NULL) {
            return -1;
        }
        arr->array = iptr;
        arr->size *= GROWTH_FACTOR;
    }
    arr->array[arr->upto] = i;
    return arr->upto++;
    
        
}

int get(DArray* arr, int index) {
    if (index >= arr->upto || index < 0) {
        return -1;
    }
    return arr->array[index];
}

int get_index(DArray* arr, int value) {
    int i;
    for (i=0; i< arr->upto; i++){
        if (arr->array[i]==value) {
            return i;
        }
    }
    return -1;
}

void DArray_free(DArray *arr) {
  free(arr->array);
}

Overwriting dynamic_array.c


In [349]:
%%file dademo.c

#include <stdio.h>
#include "dynamic_array.h"

int main() {
    DArray dynarray;
    DArray_init(&dynarray);
    printf("%d %d\n", dynarray.upto, dynarray.size);
    int i;
    for (i = 0; i < 200; i++) {
        append(&dynarray, i);
        printf("upto %d size %d val %d\n", dynarray.upto, dynarray.size, get(&dynarray, i));
    }

    for (i = 180; i < 195; i++) {
        printf("val %d index %d\n", i, get_index(&dynarray, i));
    }
    DArray_free(&dynarray);
}

Overwriting dademo.c


In [350]:
!gcc -o dademo dademo.c dynamic_array.c

In [351]:
!./dademo

0 1
upto 1 size 1 val 0
upto 2 size 2 val 1
upto 3 size 4 val 2
upto 4 size 4 val 3
upto 5 size 8 val 4
upto 6 size 8 val 5
upto 7 size 8 val 6
upto 8 size 8 val 7
upto 9 size 16 val 8
upto 10 size 16 val 9
upto 11 size 16 val 10
upto 12 size 16 val 11
upto 13 size 16 val 12
upto 14 size 16 val 13
upto 15 size 16 val 14
upto 16 size 16 val 15
upto 17 size 32 val 16
upto 18 size 32 val 17
upto 19 size 32 val 18
upto 20 size 32 val 19
upto 21 size 32 val 20
upto 22 size 32 val 21
upto 23 size 32 val 22
upto 24 size 32 val 23
upto 25 size 32 val 24
upto 26 size 32 val 25
upto 27 size 32 val 26
upto 28 size 32 val 27
upto 29 size 32 val 28
upto 30 size 32 val 29
upto 31 size 32 val 30
upto 32 size 32 val 31
upto 33 size 64 val 32
upto 34 size 64 val 33
upto 35 size 64 val 34
upto 36 size 64 val 35
upto 37 size 64 val 36
upto 38 size 64 val 37
upto 39 size 64 val 38
upto 40 size 64 val 39
upto 41 size 64 val 40
upto 42 size 64 val 41
upto 43 size 64 val 42
upto 44 size 64 val 43
upto 45 siz

### Linked Lists

Lets think of how a list with a pointer to its next element might be constructed in python

#### Nested Pairs

![](http://wla.berkeley.edu/~cs61a/fa11/lectures/img/pair.png)

(from stanford cs61a, http://wla.berkeley.edu/~cs61a/fa11/lectures/objects.html#nested-pairs)

This way to visualize a pair is called the **box and pointer** notation. It can be used to construct many abstractions, including trees.

The diagram above can be represented in python as:

In [ ]:
pair = (1,2)

This is actually not a bad representation of what happens in python as the integers are "heap" allocated, so it is some kind of pointing to the integer we need. But this representation lacks a certain power. What if we used this:

In [3]:
pair = (1, (2, None))

Which can generalize to something which looks like this:

![](http://wla.berkeley.edu/~cs61a/fa11/lectures/img/sequence.png)

In [4]:
linked_list = (1, (2, (3, (4, None))))

In [81]:
from IPython.display import HTML
HTML('<iframe width="1000" height="800" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=linked_list+%3D+(1,+(2,+(3,+(4,+None%29%29%29%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=true&textReferences=false&py=2&rawInputLstJSON=%5B%5D&curInstr=1&codeDivWidth=350&codeDivHeight=400"> </iframe>')

The ability for tuples to nest like so is called the *closure* property of tuples, not to be confused with the language construct closures. Closure property is key to representiang hierarchical data structures such as linked lists and trees, where the structures are made up of parts which are recursively composed of other parts. We will see such structures later, such as heaps, binary search trees, LSM-trees, and b-trees.

Why might you want to use a linked list like the example above?

- you allocate memory only when you want to use it
- inserting a new element is cheaper than in a fixed size array
- python arrays allocate space in certain sizes, linked lists may or may not beat them for this depending on whether you are at a memory boundary and the size of your problem.
- linked lists are useful in defining stacks, fifo's and queues (but so are regular srrays: once again this allows size mobility).

But mostly for us, its our gateway drug to other "pointer"ish and hierarchical structures.


#### quick linked list implementation

We write a very quick and dirty implementation: it does not handle edge cases like no element, but serves to illustrate the idea.

In [356]:
emptyll = None

def make_ll(first, rest):
        return (first, rest)
def first(ll):
        return ll[0]
def rest(ll):
        return ll[1]

In [357]:
myll = make_ll(1,make_ll(2, make_ll(3,emptyll)))
myll

(1, (2, (3, None)))

In [358]:
print(first(myll), "|", rest(myll), "|", first(rest(myll)))

1 | (2, (3, None)) | 2


In [359]:
def len_ll(ll):
    count =0
    curr_pointer=ll
    while curr_pointer is not emptyll:
        curr_pointer = rest(curr_pointer)
        count = count +1
    return count
len_ll(myll)

3

In [362]:
def getitem_ll(ll, i):
        while i > 0:
            ll, i = rest(ll), i - 1
        return first(ll)
getitem_ll(myll,2)

3

In [361]:
def getindex_ll(ll,v):
    count =0
    curr_pointer=ll
    while curr_pointer is not emptyll:
        if first(curr_pointer) == v:
            return count
        curr_pointer = rest(curr_pointer)
        count = count +1
    return count
getindex_ll(myll, 3)

2

### Linked List vs Array Complexities

From this implementation we can see some things:

- Insertion at the front of a linked list is O(1)(just wrap with one more `make_ll`. Insertion at the back is O(n). (because you must traverse through to find `emptyll`). Arrays or dynamic arrays cost O(n) (amortized for the latter to append to)
- Searching a linked list(`getindex_ll`) is like linear search on an array: O(n)
- What about deletion? See the lab.
- What about indexing? Its O(1) for arrays (you just seek). But in linked lists its O(n) as you must follow the pointers (`getitem_ll`).
- Setting a value (not implemented here) follows the indexing complexity.

The space complexity of both arrays and lists is O(n).

The Linked list has some pluses:

- memory wont overflow, you just allocate more
- insertion and deletion (especially at end) is rather simple, and dousent come with attendant resizing or unuse worries.
- if the records (here ints) are themselves large (say they are heap allocated C structs or complex python objects), the memory impact could actually be lower

But we have given up efficient random access and cache locality (for the smaller records anyways).

## Abstract Data Types: Stacks and Queues

Stacks (lifo) and Queues (fifo) are examples of Data Structures that can be defined on top of arrays or linked lists.  Here we'll briefly talk about stacks.

Stacks and Queues are examples of Abstract Data Types. From [Wikipedia](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=2&ved=0ahUKEwjAtOrD19XKAhXDGR4KHcjiBjYQFggfMAE&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FAbstract_data_type&usg=AFQjCNHuRIHbjbVgiNkX9FswBAbI6gMxww&sig2=5eYQGrDu606gM-S-4ays5Q):

>In computer science, an abstract data type (ADT) is a mathematical model for data types where a data type is defined by its behavior (semantics) from the point of view of a user of the data, specifically in terms of possible values, possible operations on data of this type, and the behavior of these operations.

For a Stack the two operations are
- `push(s, item)`
- `item = pop(s)`

Sometimes we might want to support a `peek` operation: `peek(s)` which merely peeks at the top item on the stack rather than popping it.

Stacks are useful in dealing with function calls, environment frames, parsing, and a whole nunch of other applications.

One way to implement Stacks is to simply use Python lists. These, remember, are dynamic arrays, so we can support arbitrarily large stacks. Addition to the stack is an amortized O(n). Deletion is O(n) if we must move all things down by 1. Else just O(1) (one could use an inactive flag).

In [251]:
astack = [1,2,4,5]
astack.append(9)
astack

[1, 2, 4, 5, 9]

In [252]:
astack.pop()

9

In [255]:
astack

[1, 2, 4, 5]

Another way to implement a stack is a linked list. You would "push" things onto the front in O(1) and pop them off in O(1) as well.

In [371]:
import reprlib
class Stack:
    
    def __init__(self, initvalue):
        self._stack = make_ll(initvalue, emptyll)
        
    def __repr__(self):
        return "Stack("+reprlib.repr(self._stack)+")"
    
    def push(self, item):
        self._stack = make_ll(item, self._stack)
        
    def pop(self):
        popped = first(self._stack)
        self._stack = rest(self._stack)
        return popped
        
    def peek(self):
        return first(self._stack)

In [372]:
s = Stack(-1)# using -1 as a sentinel value

In [373]:
s.push(1)
s.push(2)
print(s)
s.pop()

Stack((2, (1, (-1, None))))


2

In [374]:
s

Stack((1, (-1, None)))

In [375]:
s.peek()

1

For next time, think about what the advantages and tradeoof of using one implementation over another are.